In [ ]:
### using dash
import plotly.express as px
import pandas as pd
import base64
from PIL import Image
import dash
from dash import dcc, html, Input, Output

#_______________________________________data preparation
# Load your dataset
data = pd.read_csv('dataframe_allroutename.csv')
# Define a dictionary mapping scenario numbers to colors
scenario_type = {
    1: 'no_adaptive & low_traffic',
    2: 'adaptive & low_traffic',
    3: 'no_adaptive & high_traffic',
    4: 'adaptive & high_traffic'
}

# Add a new column 'scenario_type' based on the 'scenario' column
data['scenario_type'] = data['scenario'].map(scenario_type)

# Convert 'posX' and 'posZ' columns to numerical type
data['posX'] = pd.to_numeric(data['posX'], errors='coerce')
data['posZ'] = pd.to_numeric(data['posZ'], errors='coerce')

# Load your background image and get dimensions
with open("map_vector_v3_icons.png", "rb") as background_image:
    encoded_string = base64.b64encode(background_image.read()).decode('utf-8')
    img = Image.open(background_image)
    img_width, img_height = img.size

##_______________________________________create dash
app = dash.Dash(__name__)

# Define dropdown options based on unique scenarios
scenario_dropdown_options = [{'label': f'Scenario {scenario}', 'value': scenario} for scenario in data['scenario'].unique()]

# Define dropdown options based on unique start points
start_point_dropdown_options = [{'label': f'Start Point {point}', 'value': point} for point in data['start_point'].unique()]

# Define dropdown options based on unique route names
cleaned_data = data.dropna(subset=['routename'])
route_names = cleaned_data['routename'].unique()
route_dropdown_options = [{'label': name, 'value': name} for name in route_names]


# _________________________app layout
app.layout = html.Div([
    dcc.Dropdown(
        id='scenario-dropdown',
        options=scenario_dropdown_options,
        value=None,  # Default value is the first scenario in the dataset
        style={'width': '50%'},
        multi=True,
    ),
    dcc.Dropdown(
        id='start-point-dropdown',
        options=start_point_dropdown_options,
        value=None,  # Default value is the first start point in the dataset
        style={'width': '50%'},
        multi=True,
    ),
    dcc.Dropdown(
        id='route-dropdown',
        options=route_dropdown_options,
        value=None,  # Default value is the first route name in the dataset
        style={'width': '50%'},
        multi=True,
    ),
    dcc.RadioItems(
        id='highlight-points',
        options=[
            {'label': 'Highlight map interaction points', 'value': 'highlight'},
            {'label': 'Do not highlight', 'value': 'none'}
        ],
        value='none',
        labelStyle={'display': 'block'}
    ),
    html.Div(dcc.Graph(
        id='output-graph',
        style={'height': '1300px'}  # Adjust height here
    )),
])




# _________________________callback
@app.callback(
    Output('output-graph', 'figure'),
    [Input('scenario-dropdown', 'value'),
     Input('start-point-dropdown', 'value'),
     Input('route-dropdown', 'value'),
     Input('highlight-points', 'value')]
)
def update_graph(selected_scenarios, selected_start_points, selected_routes, highlight):
    # Initialize filtered data with the entire dataset
    filtered_data = data.copy()

    # Filter data based on selected scenarios, start points, and route names if they are not empty
    if selected_scenarios:
        filtered_data = filtered_data[filtered_data['scenario'].isin(selected_scenarios)]
    if selected_start_points:
        filtered_data = filtered_data[filtered_data['start_point'].isin(selected_start_points)]
    if selected_routes:
        filtered_data = filtered_data[filtered_data['routename'].isin(selected_routes)]

    # Create a scatter plot with background image
    fig = px.scatter(filtered_data, x='posX', y='posZ', title='Position Scatter Plot', color='scenario_type')

    # Highlight map interaction points based on their phases
    if highlight == 'highlight':    
        # Update marker colors based on mapInteractions
        marker_colors = ['red' if pd.notna(interaction) and interaction != "NaN" else 'grey' for interaction in filtered_data['mapInteractions']]
        fig.update_traces(marker=dict(color=marker_colors))

    # Adjust the size of points
    fig.update_traces(marker=dict(size=4))

    # Add background image
    fig.update_layout(images=[dict(
        source=f'data:image/ext;base64,{encoded_string}',
        xref="x",
        yref="y",
        x=-1349.39,
        y=-217.9444,
        sizex=1510.5924,  # Set sizex to the width of the image
        sizey=1437.8156,  # Set sizey to the height of the image
        sizing="stretch",
        opacity=1,
        layer="below")])

    # Return the plot
    return fig



   


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
# chenge the layout
### using dash
import plotly.express as px
import pandas as pd
import base64
from PIL import Image
import dash
from dash import dcc, html, Input, Output

#_______________________________________data preparation
# Load your dataset
data = pd.read_csv('dataframe_allroutename.csv')
# Define a dictionary mapping scenario numbers to colors
scenario_type = {
    1: 'no_adaptive & low_traffic',
    2: 'adaptive & low_traffic',
    3: 'no_adaptive & high_traffic',
    4: 'adaptive & high_traffic'
}

# Add a new column 'scenario_type' based on the 'scenario' column
data['scenario_type'] = data['scenario'].map(scenario_type)

# Convert 'posX' and 'posZ' columns to numerical type
data['posX'] = pd.to_numeric(data['posX'], errors='coerce')
data['posZ'] = pd.to_numeric(data['posZ'], errors='coerce')

# Load your background image and get dimensions
with open("map_vector_v3_icons.png", "rb") as background_image:
    encoded_string = base64.b64encode(background_image.read()).decode('utf-8')
    img = Image.open(background_image)
    img_width, img_height = img.size

##_______________________________________create dash
app = dash.Dash(__name__)

# Define dropdown options based on unique scenarios
scenario_dropdown_options = [{'label': f'Scenario {scenario}', 'value': scenario} for scenario in data['scenario'].unique()]

# Define dropdown options based on unique start points
start_point_dropdown_options = [{'label': f'Start Point {point}', 'value': point} for point in data['start_point'].unique()]

# Define dropdown options based on unique route names
cleaned_data = data.dropna(subset=['routename'])
route_names = cleaned_data['routename'].unique()
route_dropdown_options = [{'label': name, 'value': name} for name in route_names]


# _________________________app layout
# Define app layout
app.layout = html.Div([
    html.Div([
        html.Label('Scenario', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='scenario-dropdown',
            options=scenario_dropdown_options,
            value=None,  # Default value is the first scenario in the dataset
            style={'width': '150%'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '10px'}),  # Add margin between dropdowns
    
    html.Div([
        html.Label('Start Point', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='start-point-dropdown',
            options=start_point_dropdown_options,
            value=None,  # Default value is the first start point in the dataset
            style={'width': '150%'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '300px'}),  # Add margin between dropdowns

    html.Div([
        html.Label('Route', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='route-dropdown',
            options=route_dropdown_options,
            value=None,  # Default value is the first route name in the dataset
            style={'width': '200%', 'whiteSpace': 'nowrap'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '600px'}),  # Add margin between dropdowns

    html.Div([
            html.Label('Highlight Points', style={'font-weight': 'bold'}),
            dcc.RadioItems(
                id='highlight-points',
                options=[
                    {'label': 'Highlight map interaction points', 'value': 'highlight'},
                    {'label': 'Do not highlight', 'value': 'none'}
                ],
                value='none',
                labelStyle={'display': 'block'}
            ),
        ], style={'position': 'absolute', 'top': '10px', 'left': '1000px'}),  # Adjust position here


    html.Div(dcc.Graph(
        id='output-graph',
        style={'height': '900px'}  # Adjust height here
    )),
], style={'margin': '80px'})  # Adjust margin to create space around the layout





# _________________________callback
@app.callback(
    Output('output-graph', 'figure'),
    [Input('scenario-dropdown', 'value'),
     Input('start-point-dropdown', 'value'),
     Input('route-dropdown', 'value'),
     Input('highlight-points', 'value')]
)
def update_graph(selected_scenarios, selected_start_points, selected_routes, highlight):
    # Initialize filtered data with the entire dataset
    filtered_data = data.copy()

    # Filter data based on selected scenarios, start points, and route names if they are not empty
    if selected_scenarios:
        filtered_data = filtered_data[filtered_data['scenario'].isin(selected_scenarios)]
    if selected_start_points:
        filtered_data = filtered_data[filtered_data['start_point'].isin(selected_start_points)]
    if selected_routes:
        filtered_data = filtered_data[filtered_data['routename'].isin(selected_routes)]

    # Create a scatter plot with background image
    fig = px.scatter(filtered_data, x='posX', y='posZ', title='Position Scatter Plot', color='scenario_type')

    # Highlight map interaction points based on their phases
    if highlight == 'highlight':    
        # Update marker colors based on mapInteractions
        marker_colors = ['red' if pd.notna(interaction) and interaction != "NaN" else 'grey' for interaction in filtered_data['mapInteractions']]
        fig.update_traces(marker=dict(color=marker_colors))

    # Adjust the size of points
    fig.update_traces(marker=dict(size=4))

    # Add background image
    fig.update_layout(images=[dict(
        source=f'data:image/ext;base64,{encoded_string}',
        xref="x",
        yref="y",
        x=-1349.39,
        y=-217.9444,
        sizex=1510.5924,  # Set sizex to the width of the image
        sizey=1437.8156,  # Set sizey to the height of the image
        sizing="stretch",
        opacity=1,
        layer="below")])

    # Return the plot
    return fig



   


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [10]:
# aviod distorted 
# chenge the layout
### using dash
import plotly.express as px
import pandas as pd
import base64
from PIL import Image
import dash
from dash import dcc, html, Input, Output

#_______________________________________data preparation
# Load your dataset
data = pd.read_csv('df_all.csv')
# Define a dictionary mapping scenario numbers to colors
scenario_type = {
    1: 'no_adaptive & low_traffic',
    2: 'adaptive & low_traffic',
    3: 'no_adaptive & high_traffic',
    4: 'adaptive & high_traffic'
}

# Add a new column 'scenario_type' based on the 'scenario' column
data['scenario_type'] = data['scenario'].map(scenario_type)

# Convert 'posX' and 'posZ' columns to numerical type
data['posX'] = pd.to_numeric(data['posX'], errors='coerce')
data['posZ'] = pd.to_numeric(data['posZ'], errors='coerce')

# Load your background image and get dimensions
with open("map_vector_v3_icons.png", "rb") as background_image:
    encoded_string = base64.b64encode(background_image.read()).decode('utf-8')
    img = Image.open(background_image)
    img_width, img_height = img.size

##_______________________________________create dash
app = dash.Dash(__name__)

# Define dropdown options based on unique scenarios
scenario_dropdown_options = [{'label': f'Scenario {scenario}', 'value': scenario} for scenario in data['scenario'].unique()]

# Define dropdown options based on unique start points
start_point_dropdown_options = [{'label': f'Start Point {point}', 'value': point} for point in data['start_point'].unique()]

# Define dropdown options based on unique route names
cleaned_data = data.dropna(subset=['routename'])
route_names = cleaned_data['routename'].unique()
route_dropdown_options = [{'label': name, 'value': name} for name in route_names]


# _________________________app layout
# Define app layout
app.layout = html.Div([
    html.Div([
        html.Label('Scenario', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='scenario-dropdown',
            options=scenario_dropdown_options,
            value=None,  # Default value is the first scenario in the dataset
            style={'width': '150%'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '10px', 'z-index': '1'}),  # Add margin between dropdowns
    
    html.Div([
        html.Label('Start Point', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='start-point-dropdown',
            options=start_point_dropdown_options,
            value=None,  # Default value is the first start point in the dataset
            style={'width': '150%'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '300px', 'z-index': '1'}),  # Add margin between dropdowns

    html.Div([
        html.Label('Route', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='route-dropdown',
            options=route_dropdown_options,
            value=None,  # Default value is the first route name in the dataset
            style={'width': '200%', 'whiteSpace': 'nowrap'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '600px', 'z-index': '1'}),  # Add margin between dropdowns

    html.Div([
            html.Label('Highlight Points', style={'font-weight': 'bold'}),
            dcc.RadioItems(
                id='highlight-points',
                options=[
                    {'label': 'Highlight map interaction points', 'value': 'highlight'},
                    {'label': 'Do not highlight', 'value': 'none'}
                ],
                value='none',
                labelStyle={'display': 'block'}
            ),
        ], style={'position': 'absolute', 'top': '10px', 'left': '1000px', 'z-index': '1'}),  # Adjust position here


    html.Div(dcc.Graph(
        id='output-graph',
        style={'height': '1300px'}  # Adjust height here
    )),
], style={'margin': '20px', 'z-index': '0'})  # Adjust margin to create space around the layout





# _________________________callback
# _________________________callback
@app.callback(
    Output('output-graph', 'figure'),
    [Input('scenario-dropdown', 'value'),
     Input('start-point-dropdown', 'value'),
     Input('route-dropdown', 'value'),
     Input('highlight-points', 'value')]
)
def update_graph(selected_scenarios, selected_start_points, selected_routes, highlight):
    # Initialize filtered data with the entire dataset
    filtered_data = data.copy()

    # Filter data based on selected scenarios, start points, and route names if they are not empty
    if selected_scenarios:
        filtered_data = filtered_data[filtered_data['scenario'].isin(selected_scenarios)]
    if selected_start_points:
        filtered_data = filtered_data[filtered_data['start_point'].isin(selected_start_points)]
    if selected_routes:
        filtered_data = filtered_data[filtered_data['routename'].isin(selected_routes)]

    # Create a scatter plot with background image
    fig = px.scatter(filtered_data, x='posX', y='posZ', color='scenario_type')

    if highlight == 'highlight':    
    # Update marker colors based on mapInteractions
        marker_colors = ['red' if pd.notna(interaction) and interaction != "NaN" else 'grey' for interaction in filtered_data['mapInteractions']]
        fig.update_traces(marker=dict(color=marker_colors, size=[6 if color == 'red' else 4 for color in marker_colors]))
    else:
    # Adjust the size of all points
        fig.update_traces(marker=dict(size=3))

        
        

    # Add background image
    fig.update_layout(images=[dict(
        source=f'data:image/ext;base64,{encoded_string}',
        xref="x",
        yref="y",
        x=-1349.39,
        y=-217.9444,
        sizex=1510.5924,  # Set sizex to the width of the image
        sizey=1437.8156,  # Set sizey to the height of the image
        sizing="stretch",
        opacity=1,
        layer="below")])

    # Set fixed ranges for x and y axess
    fig.update_layout(xaxis=dict(range=[-1349.39, 161.2027 ]), yaxis=dict(range=[-1655.76, -217.9444]))

    # Return the plot
    return fig




# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [12]:
# chenge the highlight points
### using dash
import plotly.express as px
import pandas as pd
import base64
from PIL import Image
import dash
from dash import dcc, html, Input, Output

#_______________________________________data preparation
# Load your dataset
data = pd.read_csv('df_lockmark.csv')
# Define a dictionary mapping scenario numbers to colors
scenario_type = {
    1: 'no_adaptive & low_traffic',
    2: 'adaptive & low_traffic',
    3: 'no_adaptive & high_traffic',
    4: 'adaptive & high_traffic'
}

# Add a new column 'scenario_type' based on the 'scenario' column
data['scenario_type'] = data['scenario'].map(scenario_type)

# Convert 'posX' and 'posZ' columns to numerical type
data['posX'] = pd.to_numeric(data['posX'], errors='coerce')
data['posZ'] = pd.to_numeric(data['posZ'], errors='coerce')

# Load your background image and get dimensions
with open("map_vector_v3_icons.png", "rb") as background_image:
    encoded_string = base64.b64encode(background_image.read()).decode('utf-8')
    img = Image.open(background_image)
    img_width, img_height = img.size

##_______________________________________create dash
app = dash.Dash(__name__)

# Define dropdown options based on unique scenarios
scenario_dropdown_options = [{'label': f'Scenario {scenario}', 'value': scenario} for scenario in data['scenario'].unique()]

# Define dropdown options based on unique start points
start_point_dropdown_options = [{'label': f'Start Point {point}', 'value': point} for point in data['start_point'].unique()]

# Define dropdown options based on unique route names
cleaned_data = data.dropna(subset=['routename'])
route_names = cleaned_data['routename'].unique()
route_dropdown_options = [{'label': name, 'value': name} for name in route_names]


# _________________________app layout
# Define app layout
app.layout = html.Div([
    html.Div([
        html.Label('Scenario', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='scenario-dropdown',
            options=scenario_dropdown_options,
            value=None,  # Default value is the first scenario in the dataset
            style={'width': '150%'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '10px', 'z-index': '1'}),  # Add margin between dropdowns
    
    html.Div([
        html.Label('Start Point', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='start-point-dropdown',
            options=start_point_dropdown_options,
            value=None,  # Default value is the first start point in the dataset
            style={'width': '150%'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '300px', 'z-index': '1'}),  # Add margin between dropdowns

    html.Div([
        html.Label('Route', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='route-dropdown',
            options=route_dropdown_options,
            value=None,  # Default value is the first route name in the dataset
            style={'width': '200%', 'whiteSpace': 'nowrap'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '600px', 'z-index': '1'}),  # Add margin between dropdowns

    html.Div([
            html.Label('Highlight Points', style={'font-weight': 'bold'}),
            dcc.RadioItems(
                id='highlight-points',
                options=[
                    {'label': 'Highlight map interaction points', 'value': 'highlight'},
                    {'label': 'Do not highlight', 'value': 'none'}
                ],
                value='none',
                labelStyle={'display': 'block'}
            ),
        ], style={'position': 'absolute', 'top': '10px', 'left': '1000px', 'z-index': '1'}), 

    html.Div(dcc.Graph(
        id='output-graph',
        style={'height': '1300px'}  # Adjust height here
    )),
], style={'margin': '20px', 'z-index': '0'})  # Adjust margin to create space around the layout





# _________________________callback
# _________________________callback
@app.callback(
    Output('output-graph', 'figure'),
    [Input('scenario-dropdown', 'value'),
     Input('start-point-dropdown', 'value'),
     Input('route-dropdown', 'value'),
     Input('highlight-points', 'value')]
)
def update_graph(selected_scenarios, selected_start_points, selected_routes, highlight):
    # Initialize filtered data with the entire dataset
    filtered_data = data.copy()

    # Filter data based on selected scenarios, start points, and route names if they are not empty
    if selected_scenarios:
        filtered_data = filtered_data[filtered_data['scenario'].isin(selected_scenarios)]
    if selected_start_points:
        filtered_data = filtered_data[filtered_data['start_point'].isin(selected_start_points)]
    if selected_routes:
        filtered_data = filtered_data[filtered_data['routename'].isin(selected_routes)]

    # Create a scatter plot with background image
    fig = px.scatter(filtered_data, x='posX', y='posZ', color='scenario_type')



    if highlight == 'highlight':
    # Define interactions to highlight
        interactions_to_highlight = ['mapLog:positionXYZ', 'mapLog:zoom', 'mapLog:isScreenLocked+True', 'mapLog:MapButtonPoiClick+buttonOnFollowPlayer', 'mapLog:MapButtonPoiClick+buttonOnFocusOnPlayer']
        
        # Update marker colors based on mapInteractions
        marker_colors = []
        for interaction in filtered_data['mapInteractions']:
            if  any(interaction_to_highlight in str(interaction) for interaction_to_highlight in interactions_to_highlight):
                marker_colors.append('red')
            else:
                marker_colors.append('grey')
        print(marker_colors)  # Debugging statement to check marker colors
    
        fig.update_traces(marker=dict(color=marker_colors, size=[6 if color == 'red' else 4 for color in marker_colors]))
    else:
    # Adjust the size of all points
        fig.update_traces(marker=dict(size=3))


        

    # Add background image
    fig.update_layout(images=[dict(
        source=f'data:image/ext;base64,{encoded_string}',
        xref="x",
        yref="y",
        x=-1349.39,
        y=-217.9444,
        sizex=1510.5924,  # Set sizex to the width of the image
        sizey=1437.8156,  # Set sizey to the height of the image
        sizing="stretch",
        opacity=1,
        layer="below")])

    # Set fixed ranges for x and y axess
    fig.update_layout(xaxis=dict(range=[-1349.39, 161.2027 ]), yaxis=dict(range=[-1655.76, -217.9444]))

    # Return the plot
    return fig




# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [20]:
# add screen lock points 
### using dash
import plotly.express as px
import pandas as pd
import base64
from PIL import Image
import dash
from dash import dcc, html, Input, Output

#_______________________________________data preparation
# Load your dataset
data = pd.read_csv('df_lockmark.csv')
# Define a dictionary mapping scenario numbers to colors
scenario_type = {
    1: 'no_adaptive & low_traffic',
    2: 'adaptive & low_traffic',
    3: 'no_adaptive & high_traffic',
    4: 'adaptive & high_traffic'
}

# Add a new column 'scenario_type' based on the 'scenario' column
data['scenario_type'] = data['scenario'].map(scenario_type)

# Convert 'posX' and 'posZ' columns to numerical type
data['posX'] = pd.to_numeric(data['posX'], errors='coerce')
data['posZ'] = pd.to_numeric(data['posZ'], errors='coerce')

# Load your background image and get dimensions
with open("map_vector_v3_icons.png", "rb") as background_image:
    encoded_string = base64.b64encode(background_image.read()).decode('utf-8')
    img = Image.open(background_image)
    img_width, img_height = img.size

##_______________________________________create dash
app = dash.Dash(__name__)

# Define dropdown options based on unique scenarios
scenario_dropdown_options = [{'label': f'Scenario {scenario}', 'value': scenario} for scenario in data['scenario'].unique()]

# Define dropdown options based on unique start points
start_point_dropdown_options = [{'label': f'Start Point {point}', 'value': point} for point in data['start_point'].unique()]

# Define dropdown options based on unique route names
cleaned_data = data.dropna(subset=['routename'])
route_names = cleaned_data['routename'].unique()
route_dropdown_options = [{'label': name, 'value': name} for name in route_names]


# _________________________app layout
# Define app layout
app.layout = html.Div([
    html.Div([
        html.Label('Scenario', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='scenario-dropdown',
            options=scenario_dropdown_options,
            value=None,  # Default value is the first scenario in the dataset
            style={'width': '150%'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '10px', 'z-index': '1'}),  # Add margin between dropdowns
    
    html.Div([
        html.Label('Start Point', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='start-point-dropdown',
            options=start_point_dropdown_options,
            value=None,  # Default value is the first start point in the dataset
            style={'width': '150%'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '300px', 'z-index': '1'}),  # Add margin between dropdowns

    html.Div([
        html.Label('Route', style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='route-dropdown',
            options=route_dropdown_options,
            value=None,  # Default value is the first route name in the dataset
            style={'width': '200%', 'whiteSpace': 'nowrap'},
            multi=True,
        ),
    ], style={'position': 'absolute', 'top': '10px', 'left': '600px', 'z-index': '1'}),  # Add margin between dropdowns

    html.Div([
            html.Label('Highlight Points', style={'font-weight': 'bold'}),
            dcc.RadioItems(
                id='highlight-points',
                options=[
                    {'label': 'Highlight map interaction points', 'value': 'highlight'},
                    {'label': 'Do not highlight', 'value': 'none'}
                ],
                value='none',
                labelStyle={'display': 'block'}
            ),
        ], style={'position': 'absolute', 'top': '10px', 'left': '900px', 'z-index': '1'}), 

    html.Div([
            html.Label('Lockscreen Points', style={'font-weight': 'bold'}),
            dcc.RadioItems(
                id='lockscreen-points',
                options=[
                    {'label': 'Highlight lockscreen points', 'value': 'highlight'},
                    {'label': 'Do not highlight', 'value': 'none'}
                ],
                value='none',
                labelStyle={'display': 'block'}
            ),
        ], style={'position': 'absolute', 'top': '10px', 'left': '1200px', 'z-index': '1'}),  # Adjust position here



    html.Div(dcc.Graph(
        id='output-graph',
        style={'height': '1300px'}  # Adjust height here
    )),
], style={'margin': '20px', 'z-index': '0'})  # Adjust margin to create space around the layout






# _________________________callback
@app.callback(
    Output('output-graph', 'figure'),
    [Input('scenario-dropdown', 'value'),
     Input('start-point-dropdown', 'value'),
     Input('route-dropdown', 'value'),
     Input('highlight-points', 'value'),
     Input('lockscreen-points', 'value')]
)
def update_graph(selected_scenarios, selected_start_points, selected_routes, highlight, lockscreen_highlight):
    # Initialize filtered data with the entire dataset
    filtered_data = data.copy()

    # Filter data based on selected scenarios, start points, and route names if they are not empty
    if selected_scenarios:
        filtered_data = filtered_data[filtered_data['scenario'].isin(selected_scenarios)]
    if selected_start_points:
        filtered_data = filtered_data[filtered_data['start_point'].isin(selected_start_points)]
    if selected_routes:
        filtered_data = filtered_data[filtered_data['routename'].isin(selected_routes)]

    # Create a scatter plot with background image
    fig = px.scatter(filtered_data, x='posX', y='posZ', color='scenario_type')



    if highlight == 'highlight':
    # Define interactions to highlight
        interactions_to_highlight = ['mapLog:positionXYZ', 'mapLog:zoom', 'mapLog:isScreenLocked+True', 'mapLog:MapButtonPoiClick+buttonOnFollowPlayer', 'mapLog:MapButtonPoiClick+buttonOnFocusOnPlayer']
        
        # Update marker colors based on mapInteractions
        marker_colors = []
        for interaction in filtered_data['mapInteractions']:
            if  any(interaction_to_highlight in str(interaction) for interaction_to_highlight in interactions_to_highlight):
                marker_colors.append('red')
            else:
                marker_colors.append('grey')
        print(marker_colors)  # Debugging statement to check marker colors
    
        fig.update_traces(marker=dict(color=marker_colors, size=[6 if color == 'red' else 4 for color in marker_colors]))

    elif lockscreen_highlight == 'highlight':  
        # Update marker colors based on lockscreen column
        marker_colors = ['blue' if lock == 1 else 'grey' for lock in filtered_data['lockscreen']]
        fig.update_traces(marker=dict(color=marker_colors, size=[6 if color == 'blue' else 4 for color in marker_colors]))
    
    else:
        # Adjust the size of all points
        fig.update_traces(marker=dict(size=3))

        

    # Add background image
    fig.update_layout(images=[dict(
        source=f'data:image/ext;base64,{encoded_string}',
        xref="x",
        yref="y",
        x=-1349.39,
        y=-217.9444,
        sizex=1510.5924,  # Set sizex to the width of the image
        sizey=1437.8156,  # Set sizey to the height of the image
        sizing="stretch",
        opacity=1,
        layer="below")])

    # Set fixed ranges for x and y axess
    fig.update_layout(xaxis=dict(range=[-1349.39, 161.2027 ]), yaxis=dict(range=[-1655.76, -217.9444]))

    # Return the plot
    return fig




# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


['grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'red', 'red', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'red', 'grey', 'red', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'red', 'grey', 'grey', 'grey', 'red', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'red', 'grey', 'red', 'grey', 'grey', 'grey', 'red', 'red', 'red', 'grey', 'grey', 'grey', 'red', 'grey', 'red', 'grey', 'grey', 'red', 'grey', 'red', 'grey', 'grey', 'red', 'grey', 'red', 'grey', 'grey', 'grey', 'red', 'red', 'grey', 'grey', 'red', 'grey', 'red', 'grey', 'red', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey